In [1]:
import os
import time
import pickle
import shutil
import random
import urllib
import requests
import concurrent.futures
from  urllib.request import urlopen
from termcolor import colored
from utils import Config, safe_pickle_dump

printgreen = lambda q: print(colored(q, 'green'))
printred = lambda q: print(colored(q, 'red'))

session = requests.session()
session.proxies = {
    'http': 'socks5://localhost:9050',
    'https': 'socks5://localhost:9050'
}
headers = {'User-agent': 'HotJava/1.1.2 FCS'}

In [2]:
req = session.get('https://api.ipify.org', timeout = 15)
req.content
#with open(fname, 'wb') as fp:
#shutil.copyfileobj(req.raw, fp)

b'62.210.105.116'

In [3]:
db = pickle.load(open(Config.db_path, 'rb'))

In [4]:
have = set(os.listdir(Config.pdf_dir))

In [5]:
print(len(db))
print(len(have))

print(len(db) - len(have))

116864
115844
1020


In [6]:
have = set(os.listdir(Config.pdf_dir))

def refreshNeeded():
    entries = []
    for db_key, j in db.items():
        pdfs = [x['href'] for x in j['links'] if x['type'] == 'application/pdf']
        assert len(pdfs) == 1
        if ('ishtml' in db[db_key] and db[db_key]['ishtml'] == True):
            #print(db_key)
            continue
        pdf_url = pdfs[0] + '.pdf'
        basename = pdf_url.split('/')[-1]
        fname = os.path.join(Config.pdf_dir, basename)
        if not basename in have:
            entries.append({
                'key': db_key, 
                'url': pdf_url, 
                'basename': basename, 
                'filename': fname
            })
    return entries

newEntries = refreshNeeded()

print(len(newEntries))

1397


In [7]:
print(newEntries[0])
print(list(have)[0])

{'key': '1802.00488', 'url': 'http://arxiv.org/pdf/1802.00488v3.pdf', 'basename': '1802.00488v3.pdf', 'filename': 'data/pdf/1802.00488v3.pdf'}
0310162v1.pdf


In [8]:
def load_url(pdf_url):
    return session.get(pdf_url, timeout = 30)

def reqToFile(db_key, basename, filename, req):
    if (req.ok and req.headers['Content-Type'] == 'application/pdf'):
        with open(filename, 'wb') as f:
            f.write(req.content)
        print(f"{filename} written")
    elif ('text/html' in req.headers['Content-Type']):
        db[db_key]['ishtml'] = True
        raise ValueError(f"{db_key}: was HTML instead of PDF")
    else:
        raise Exception(f"{db_key}: Unknown error")

In [10]:
have = set(os.listdir(Config.pdf_dir))
newEntries = refreshNeeded()

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    
    future_to_url = { executor.submit(load_url, entry['url']): entry for entry in newEntries }
    
    for future in concurrent.futures.as_completed(future_to_url):
        try:
            this_entry = future_to_url[future]
            print(f"Trying {this_entry['key']}")
            req = future.result()
            reqToFile(this_entry['key'], this_entry['basename'], this_entry['filename'], req)
        except Exception as e:
            print(e)
safe_pickle_dump(db, Config.db_path)
print("Done")

Trying 1803.00203
data/pdf/1803.00203v3.pdf written
Trying 1712.02644
data/pdf/1712.02644v2.pdf written
Trying 1807.05709
data/pdf/1807.05709v2.pdf written
Trying 1710.09723
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Trying 1705.00023
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Trying 1711.03693
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Trying 1710.09206
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Trying 1802.03956
data/pdf/1802.03956v3.pdf written
Trying 1802.08381
data/pdf/1802.08381v2.pdf written
Trying 1712.04212
data/pdf/1712.04212v4.pdf written
Trying 1601.06472
data/pdf/1601.06472v3.pdf written
Trying 1706.01664
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Trying 1705.10857
('Connection aborted.', RemoteDisconnected('Remote end closed c

Trying 1605.00342
data/pdf/1605.00342v3.pdf written
Trying 1602.03009
data/pdf/1602.03009v2.pdf written
Trying 1512.08647
1512.08647: was HTML instead of PDF
Trying 1512.08847
data/pdf/1512.08847v2.pdf written
Trying 1604.02381
data/pdf/1604.02381v2.pdf written
Trying 1602.08593
data/pdf/1602.08593v2.pdf written
Trying 1601.06136
data/pdf/1601.06136v2.pdf written
Trying 1512.01447
data/pdf/1512.01447v2.pdf written
Trying 1606.00422
data/pdf/1606.00422v2.pdf written
Trying 1602.01189
data/pdf/1602.01189v2.pdf written
Trying 1511.07410
data/pdf/1511.07410v4.pdf written
Trying 1601.01273
data/pdf/1601.01273v3.pdf written
Trying 1503.08593
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Trying 1501.02916
1501.02916: was HTML instead of PDF
Trying 1012.1849
data/pdf/1012.1849v3.pdf written
Trying 1510.05146
1510.05146: was HTML instead of PDF
Trying 1511.00650
data/pdf/1511.00650v2.pdf written
Trying 1512.05190
data/pdf/1512.05190v2.pdf written
T

Trying 1808.01508
data/pdf/1808.01508v1.pdf written
Trying 1808.01536
data/pdf/1808.01536v1.pdf written
Trying 1808.01579
data/pdf/1808.01579v1.pdf written
Trying 1808.01179
data/pdf/1808.01179v1.pdf written
Trying 1808.01380
data/pdf/1808.01380v1.pdf written
Trying 1808.00707
data/pdf/1808.00707v1.pdf written
Trying 1807.11030
data/pdf/1807.11030v1.pdf written
Trying 1808.00715
data/pdf/1808.00715v1.pdf written
Trying 1808.00663
data/pdf/1808.00663v1.pdf written
Trying 1808.01547
data/pdf/1808.01547v1.pdf written
Trying 1808.01057
data/pdf/1808.01057v1.pdf written
Trying 1808.00602
data/pdf/1808.00602v1.pdf written
Trying 1807.10883
data/pdf/1807.10883v1.pdf written
Trying 1808.01164
data/pdf/1808.01164v1.pdf written
Trying 1808.01513
data/pdf/1808.01513v1.pdf written
Trying 1807.10131
data/pdf/1807.10131v1.pdf written
Trying 1808.01589
data/pdf/1808.01589v1.pdf written
Trying 1807.10071
data/pdf/1807.10071v1.pdf written
Trying 1807.09933
data/pdf/1807.09933v1.pdf written
Trying 1807.